In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from itertools import product
from functools import partial
from multiprocessing import Pool, cpu_count
from tqdm import tqdm

In [2]:
# Importing score classes 
from aggregate_scoring import (
     CommunityTransportationOptions,
     DesirableUndesirableActivities,
     HousingNeedsCharacteristics,
     QualityEducation,
     StableCommunities
)

In [14]:
# Defining Grid Parameters
# lon_min, lon_max = -84.911059, -83.799104
# lat_min, lat_max = 33.256500, 34.412590
# step = 0.01
# lats = np.arange(lat_min, lat_max, step)
# lons = np.arange(lon_min, lon_max, step)


# Test on smaller grid (Fulton County)
# lats = np.arange(33.45, 34.19, 0.01)
# lons = np.arange(-84.85, -84.10, 0.01)


# # Approximate dekalb county box bounds 
# LAT_MIN, LAT_MAX = 33.60, 34.00
# LON_MIN, LON_MAX = -84.30, -84.00

lats = np.arange(33.60, 34.00, 0.01)
lons = np.arange(-84.30, -84.00, 0.01)


# # Approximate gwinette county box bounds 
# LAT_MIN, LAT_MAX = 33.7, 34.10
# LON_MIN, LON_MAX = -84.25, -83.75
# lats = np.arange(33.7, 34.10, 0.01)
# lons = np.arange(-84.25,-83.75, 0.01)


# Approximate cobb county box bounds 
# LAT_MIN, LAT_MAX = 33.70, 34.10
# LON_MIN, LON_MAX = -84.75, -84.35
# lats = np.arange(33.7, 34.10, 0.01)
# lons = np.arange(-84.75, -84.35, 0.01)

print(len(lats), len(lons))




40 30


In [5]:
# Load in Datasets

# --- CommunityTransportationOptions ---
df_transit = pd.read_csv("../../data/raw/scoring_indicators/community_trans_options_sites/georgia_transit_locations_with_hub.csv")

# --- DesirableUndesirableActivities ---
rural_gdf = gpd.read_file("../../data/raw/shapefiles/USDA_Rural_Housing_by_Tract_7054655361891465054/USDA_Rural_Housing_by_Tract.shp").to_crs("EPSG:4326")
rural_union = rural_gdf.unary_union 
csv_desirable = pd.read_csv("../../data/processed/scoring_indicators/desirable_undesirable_activities/desirable_activities_google_places_v3.csv")
csv_usda = pd.read_csv("../../data/raw/scoring_indicators/desirable_undesirable_activities/usda/food_access_research_atlas.csv", dtype={'CensusTract': str})
tract_shape = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp")
csv_undesirable = pd.read_csv("../../data/processed/scoring_indicators/desirable_undesirable_activities/undesirable_hsi_tri_cdr_rcra_frs_google_places.csv")

# --- QualityEducation ---
df_school = pd.read_csv("../../data/processed/scoring_indicators/quality_education_areas/Option_C_Scores_Eligibility_with_BTO.csv")
gdf_school_boundaries = [
    gpd.read_file("../../data/raw/shapefiles/quality_education/Administrative.geojson").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/APSBoundaries.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKE.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKM.json").to_crs("EPSG:4326"),
    gpd.read_file("../../data/raw/shapefiles/quality_education/DKBHS.json").to_crs("EPSG:4326")
    ]

# --- StableCommunities ---
df_indicators = pd.read_csv("../../data/processed/scoring_indicators/stable_communities/stable_communities_2024_processed_v3.csv")
shp_tract = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp").to_crs("EPSG:4326")


/var/folders/w2/jx7kgvv14cx4km6tl2rymsmw0000gn/T/ipykernel_42778/3717262508.py:8: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  rural_union = rural_gdf.unary_union


In [6]:
# Defining kwargs for scoring classes
kwargs = {
    # --- CommunityTransportationOptions ---
    "transit_df": df_transit,

    # --- DesirableUndesirableActivities ---
    "rural_gdf_unary_union": rural_union,
    "desirable_csv": csv_desirable, 
    "grocery_csv": csv_desirable,
    "usda_csv": csv_usda,
    "tract_shapefile": tract_shape,
    "undesirable_csv": csv_undesirable,

    # --- QualityEducation ---
    "school_df": df_school,
    "school_boundary_gdfs": gdf_school_boundaries,       
    "state_avg_by_year": {
        "elementary": {
            2018: 77.8,
            2019: 79.9
        },
        "middle": {
            2018: 76.2,
            2019: 77
        },
        "high": {
            2018: 75.3,
            2019: 78.8
        }
    },

    # --- StableCommunities ---
    "indicators_df": df_indicators,
    "tracts_shp": shp_tract,
    
    # # --- HousingNeedsCharacteristics ---
    # "census_tract_data": pd.read_csv("../../data/processed/scoring_indicators/housing_needs_characteristics/merged_housing_data.csv"),
    # "tracts_gdf": gpd.read_file("../../data/raw/shapefiles/HousingNeeds/tl_2020_13_tract/tl_2020_13_tract.shp").to_crs("EPSG:4326"),
    # #"revitalization_score": 4,
    # "in_qct": False  # Required for housing need eligibility
} 


In [7]:
def score_one_point(lat, lon, kwargs):
    try:
        # Scoring objects
        ct = CommunityTransportationOptions(lat, lon, **kwargs)
        dua = DesirableUndesirableActivities(lat, lon, **kwargs)
        sc = StableCommunities(lat, lon, **kwargs)
        qe = QualityEducation(lat, lon, **kwargs)

        # Calculate individual scores
        ct_score = ct.calculate_score()
        dua_score = dua.calculate_score()
        sc_score = sc.calculate_score()
        qe_score = qe.calculate_score()

        total = ct_score + dua_score + sc_score + qe_score

        return {
            "lat": lat,
            "lon": lon,
            "community_transportation_options_score": ct_score,
            "desirable_undesirable_activities_score": dua_score,
            "quality_education_areas_score": qe_score,
            "stable_communities_score": sc_score,
            "total_score": total,
            "geometry": Point(lon, lat)
        }

    except Exception as e:
        print(f"Error at (lat={lat:.4f}, lon={lon:.4f}): {e}")
        return None


In [16]:
# Use partial to bind kwargs into the function
bound_score_fn = partial(score_one_point, kwargs=kwargs)

# Create the lat/lon grid
points = list(product(lats, lons))

# Run in parallel
with ThreadPoolExecutor(max_workers=2) as executor:
    results = list(tqdm(executor.map(lambda args: bound_score_fn(*args), points), total=len(points)))
    
# Filter and store
records = [r for r in results if r is not None]

# Output GeoJSON
gdf = gpd.GeoDataFrame(records, geometry="geometry", crs="EPSG:4326")
gdf.to_file("../../data/maps/total_location_score/final_scoring_dataset_test_dekalb.geojson", driver="GeoJSON")

  9%|▉         | 109/1200 [11:02<1:39:24,  5.47s/it]

Error at (lat=33.6300, lon=-84.1200): 'float' object has no attribute 'lower'


  9%|▉         | 110/1200 [11:07<1:33:32,  5.15s/it]

Error at (lat=33.6300, lon=-84.1100): 'float' object has no attribute 'lower'


 11%|█         | 127/1200 [13:01<2:01:34,  6.80s/it]

...

 94%|█████████▍| 1125/1200 [2:14:41<33:35, 26.87s/it]

Error at (lat=33.9700, lon=-84.1600): 'float' object has no attribute 'lower'


 94%|█████████▍| 1125/1200 [2:14:46<08:59,  7.19s/it]


Error at (lat=33.9700, lon=-84.1500): 'float' object has no attribute 'lower'
Error at (lat=33.9700, lon=-84.1400): 'float' object has no attribute 'lower'


KeyboardInterrupt: 

In [ ]:
import geopandas as gpd
gdf = gpd.read_file("../../data/maps/total_location_score/LIHTC-Project_data_map_final_scoring_by_category_multiproc.geojson")

In [ ]:
gdf